#### ¿como se comportan los valores de las residencias en California?
Conocido por su diversidad geográfica y económica, su mercado inmobiliario es influenciado por una variedad de factores. Los compradores y vendedores necesitan de una visualización geográfica que describa las ubicaciones y características inmobiliarias de las residencias para una mejor perspectiva. Además, si se estaría considerando inversiones futuras y se desea evaluar la rentabilidad de un hogar, una estimación de precios de venta óptima puede llegar a traer ideas claras y decisiones convincentes.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


data = datasets.fetch_california_housing()

df = pd.DataFrame(data["data"],columns=data["feature_names"])
df["MedHouseVal"] = data["target"]
df["MedHouseVal"] = df["MedHouseVal"] * 100000

df.describe()

#### Boosting
En el proyecto 4(evaluación en riesgo de crédito) habiamos mencionado y llevado a cabo la técnica de Bagging, donde generábamos distintos metaestimadores entrenados de forma independiente y se tomaban en cuenta los resultados de cada uno. En este caso, cada metaestimador capacitará y solucionará los errores del siguiente potenciando cada vez más la precisión final, es decir que su escalabilidad es vertical en lugar de horizontal. Requieren de una hiperparametrización compleja y una tendencia, si no realiza correctamente lo anterior mencionado, al sobreajuste, es decir, modelos con una precisión alta sobre un grupos de datos en particular pero con la poca adaptación para resolver nuevas situaciones.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df[df.columns[:-1]],
                                                    df["MedHouseVal"],
                                                    test_size=0.25)

turned_parameters = {
    "n_estimators":[100,200,300,400,500],
    "subsample":[0.7,0.75,0.8,0.85,0.9],
    "max_depth":[3,4,5,6,7],
    "learning_rate":[0.2,0.3,0.4,0.5,0.55],
    "min_child_weight":[2,3,4,5,6],
    "gamma":[0,1,2,3,4]
}

xgbr_test = XGBRegressor()

# implementando Random Search CV para obtener los mejores valores de hiperparámetros
random_search = RandomizedSearchCV(xgbr_test, turned_parameters,cv=5)
random_search.fit(df[df.columns[:-1]], df["MedHouseVal"])

# asignación de los valores óptimos obtenidos por el algoritmo
xgbr = XGBRegressor(n_estimators = random_search.best_params_["n_estimators"],
                    subsample = random_search.best_params_["subsample"],
                    max_depth = random_search.best_params_["max_depth"],
                    learning_rate = random_search.best_params_["learning_rate"],
                    min_child_weight = random_search.best_params_["min_child_weight"],
                    gamma = random_search.best_params_["gamma"])

xgbr.fit(x_train, y_train)

#### Compensación bías - varianza
Conforme hemos avanzados en los problemas, definimos los principales desafíos que se interponen en la precisión y eficacia de un modelo predictivo, la realidad es que nunca se puede estar libre de errores y la mejor solución es comprender las diferentes fuentes de error nos ayudará a obtener mejores resultados. El objetivo es lograr un bías bajo y una varianza baja, a su vez, el algoritmo debe lograr un buen rendimiento de predicción, el bías frente a la varianza se refiere a la precisión frente a la consistencia, es decir, aumentar la varianza disminuye la inprecisión en promedio pero aumenta la inconsistencia mientras que aumentar el bías disminuye la inconsistencia pero aumenta la inprecisión en promedio. El punto ideal para cualquier modelo es el nivel de complejidad en donde el aumento del bías es equivalente a la reducción de la varianza, por lo tanto se requiere enconctrar un buen equilibrio entre ambos errores de manera que minimice el error total.

In [ ]:
predicts = xgbr.predict(x_test)

# Raíz cuadrada del error cuadrático medio(RMSE)
# similar a la desviación éstandar, muestran la variabiliad de los residuos en la misma unidad que los valores reales

metrics = f"RMSE: {round(root_mean_squared_error(predicts,y_test))}"
scatter = go.Figure()
scatter.add_trace(go.Scatter(x=y_test, y=predicts, mode="markers", marker_color="rgba(255,0,0,0.45)"))
scatter.update_xaxes(title_text="Valores reales")
scatter.update_yaxes(title_text="Predicciones")
scatter.update_layout(title=metrics)
scatter

#### Aprendizaje no supervisado
las observaciones no tienen una respuesta asociada que guíe el aprendizaje, uno de sus algoritmos es Kmeans que, mediante un proceso iterativo, genera sus propias etiquetas determinando grupos de datos asociables en función de sus acercamientos estadísticos. El modelo de clustering requiere de un hiperparámetro que es la número de centroides o K-means, estos funcionan como valores dispersados entre los datos cuya cercanía irá asociandolos conformando los clústers, iterativamente los centroides seguirán desplazándose por la región reasignando nuevos valores hasta lograr alcanzar la homogeneidad. El método del codo se utiliza a la hora de designar este valor numérico y dónde el la idea es visualizar en un gráfico cartesiano diferentes cantidades de centroides e inercias(distancia entre los miembros de los clústers y su centroide) y distinguir el punto medio de ambos. También existen métricas especializadas en evaluar esta clase de modelos siendo estas Coeficiente de Silueta e Índice Davies-Bouldin.

In [ ]:
clusters = []
inertias = []

for c in range(3,12):
    kmeans = KMeans(n_clusters=c).fit(df["MedHouseVal"].values.reshape((-1,1)))
    clusters.append(c)
    inertias.append(kmeans.inertia_)
    
kmeans = KMeans(n_clusters=5).fit(df["MedHouseVal"].values.reshape((-1,1)))
inertia = kmeans.inertia_

plt.plot(clusters, inertias, marker="o")
plt.text(int(str(kmeans)[-2])+0.1, inertia, "Valor del codo")
plt.grid("on")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5).fit(df["MedHouseVal"].values.reshape((-1,1)))

clusters = kmeans.labels_

df["clusters"] = clusters

range_values = np.array([])

for c in df["clusters"].sort_values().unique():
    cluster = df.loc[df["clusters"] == c,["clusters","MedHouseVal"]]
    max_value = str(cluster["MedHouseVal"].max())
    min_value = str(cluster["MedHouseVal"].min())
    range_values = np.append(range_values,min_value)
    range_values = np.append(range_values,max_value)
    
range_values = range_values.reshape((-1,2))
    
df["clusters"] = df["clusters"].replace(
    {
        0:f"0 ({range_values[0,0][:8]}$-{range_values[0,1][:8]}$)",
        1:f"1 ({range_values[1,0][:8]}$-{range_values[1,1][:8]}$)",
        2:f"2 ({range_values[2,0][:8]}$-{range_values[2,1][:8]}$)",
        3:f"3 ({range_values[3,0][:8]}$-{range_values[3,1][:8]}$)",
        4:f"4 ({range_values[4,0][:8]}$-{range_values[4,1][:8]}$)"
    })

clusters_count = df["clusters"].value_counts().reset_index()

cluster_map = make_subplots(rows=2, cols=1, subplot_titles=["Rangos de precios","Conteo de rangos"])

def make_figure(df, cluster):

    var_x = df.loc[df["clusters"] == cluster,["Longitude", "clusters"]]
    var_y = df.loc[df["clusters"] == cluster,["Latitude", "clusters"]]

    cluster_map.add_trace(go.Scatter( 
        x=var_x["Longitude"], 
        y=var_y["Latitude"], 
        mode="markers", 
        marker=dict( 
        size=9, 
        symbol="circle", 
        line=dict(width=0.5, color="white") 
        ), 
        name=cluster), row=1, col=1) 

make_figure(df, df["clusters"].unique()[0])
make_figure(df, df["clusters"].unique()[1])
make_figure(df, df["clusters"].unique()[2])
make_figure(df, df["clusters"].unique()[3])
make_figure(df, df["clusters"].unique()[4])

cluster_map.update_xaxes(row=1, col=1, range=[-125,-114], constrain="domain")
cluster_map.update_yaxes(row=1, col=1, range=[32,42], constrain="domain", scaleanchor="x", scaleratio=1)

clusters_count = df["clusters"].value_counts().reset_index()

cluster_map.add_trace(go.Bar(x=clusters_count["clusters"], y=clusters_count["count"]), row=2, col=1)

cluster_map.update_layout(height=900)

#### Dashboard que funciona como mapa geográfico descriptivo de las viviendas y muestra sus rangos de valores

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e5_body",children=[
        html.H1("Inmuebles en California ",id="title",className="e5_title"),
            dcc.Dropdown(id="dropdown",className="e5_dropdown",
                        options = [
                            {"label":"Valor de precio","value":"MedHouseVal"},
                            {"label":"Ingreso medio","value":"MedInc"},
                            {"label":"Edad media","value":"HouseAge"},
                            {"label":"Promedio de habitaciones","value":"AveRooms"},
                            {"label":"Promedio de dormitorios","value":"AveBedrms"},
                            {"label":"Población","value":"Population"},
                            {"label":"Promedio de ocupación","value":"AveOccuption"}
                        ],
                        value="MedHouseVal",
                        multi=False,
                        clearable=False),
        dcc.Graph(id="graph_1",className="e5_graph",figure={}),
        dcc.Graph(id="graph_2",className="e5_graph",figure=cluster_map)
])

@app.callback(
    Output(component_id="graph_1",component_property="figure"),
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_var):
    
    california_map = go.Figure(go.Scattermapbox(
        lat=df["Latitude"],
        lon=df["Longitude"],
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=9,
            color=df[slct_var],
            cmin=df[slct_var].min(),
            cmax=df[slct_var].max(),
            showscale=True
        )
    ))
    
    california_map.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4.8,
    mapbox_center_lat = 37.0,
    mapbox_center_lon = -119.0,
    margin={"r":0,"t":0,"l":0,"b":0}
    )
    
    return california_map
    
if __name__ == "__main__":
    app.run_server(debug=False)